- Feature Engineering 
- 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Load cleaned data
train = pd.read_csv('train_cleand.csv')
test = pd.read_csv('test_cleand.csv')

# Save target variable and IDs
y_train = train['SalePrice']
test_ids = test['Id']
train.drop(['Id', 'SalePrice'], axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

In [ ]:
# Total area of the house
train['TotalArea'] = train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalArea'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']

# Age of the house when sold
train['HouseAge'] = train['YrSold'] - train['YearBuilt']
test['HouseAge'] = test['YrSold'] - test['YearBuilt']

# Interaction term: Overall quality * Living area
train['QualLivArea'] = train['OverallQual'] * train['GrLivArea']
test['QualLivArea'] = test['OverallQual'] * test['GrLivArea']

# Drop original columns if needed (optional)
# train.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'YrSold', 'YearBuilt'], axis=1, inplace=True)